In [ ]:
%matplotlib inline

import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from PIL import Image
import sys
from sklearn.cluster import KMeans

### CT画像の拡張子をtifからpngに変更

In [ ]:
for img_num in tqdm(range(888)):
    input_file = "../Input/ORA/ORA-{0:03d}.tif".format(img_num)
    output_file = ".../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
    
    img = cv2.imread(input_file,0)
    cv2.imwrite(output_file,img)

### バイラテラルフィルタ

In [ ]:
#画像番号の指定
img_num = 661

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
output_file_bi = "../Output/Remove_Background/Bilateral/ORA-{0:03d}_Bila.png".format(img_num)
output_file_bi_th-otsu = "../Output/Remove_Background/Bilateral/ORA-{0:03d}_Bila_th-otsu.png".format(img_num)
output_file_bi_th = "../Output/Remove_Background/Bilateral/ORA-{0:03d}_Bila_th.png".format(img_num)

#画像の入力
img = cv2.imread(input_file,0)

#バイラテラルフィルタ
bi = cv2.bilateralFilter(img, 15, 20, 20)
cv2.imwrite(output_file_bi,img)

#フィルタ結果を2値化（大津法）
#th, th_img = cv2.threshold(bi, 0, 255, cv2.THRESH_OTSU)
#cv2.imwrite(output_file_bi_th-otsu,img)

#フィルタ結果を2値化（閾値を指定）
#th = 60 #閾値
#th_img = np.where(bi > th,255,0)
#cv2.imwrite(output_file_bi_th,img)

### ノンローカルミーンフィルタ

In [ ]:
#画像番号の指定
img_num = 400

#ノンローカルミーンフィルタのパラメータを指定
h_p = 7

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
output_file_NLMD = "../Output/Remove_Background/ORA-{0:03d}_NLMD_h{1:d}.png".format(img_num,h_p)
output_file_NLMD_th-otsu = "../Output/Remove_Background/ORA-{0:03d}_NLMD_h{1:d}_th-otsu.png".format(img_num,h_p)
output_file_NLMD_th = "../Output/Remove_Background/ORA-{0:03d}_NLMD_h{1:d}_th.png".format(img_num,h_p)

#画像の入力
img = cv2.imread(input_file,0)

#ノンローカルミーンフィルタ
img_NLMD = cv2.fastNlMeansDenoising(img,h = h_p)

#フィルタ結果を2値化（大津法）
#th, th_img = cv2.threshold(img_NLMD, 0, 255, cv2.THRESH_OTSU)
#cv2.imwrite(output_file_NLMD_th-otsu,img)

#フィルタ結果を2値化（閾値を指定）
#th = 60 #閾値
#th_img = np.where(img_NLMD > th,255,0)
#cv2.imwrite(output_file_NLMD_th,img)

### スケルトン化（ノンローカルミーンフィルタ）

In [ ]:
#画像番号の指定
img_num = 661

#ノンローカルミーンフィルタのパラメータを指定
h_p = 7

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
output_file_skel = "../Output/Remove_Background/ORA-{0:03d}_NLMD_h{1:d}_skel.png".format(img_num,h_p)

#画像の入力
img = cv2.imread(input_file,0)

#ノンローカルミーンフィルタ
img_NLMD = cv2.fastNlMeansDenoising(img,h=h_p)

#フィルタ結果を2値化
th, th_img = cv2.threshold(img_NLMD, 0, 255, cv2.THRESH_OTSU)
th_img = np.where(th_img==255 ,1 ,0)

# 細線化(スケルトン化) Zha84
skeleton = skeletonize(th_img)

skel_b = np.zeros((img.shape[0],img.shape[1]))
skel_b[skeleton==True]=255

cv2.imwrite(output_file_skel,skel_b)

### 分岐点・端点検出

In [ ]:
#画像番号の指定
img_num = 661

#ノンローカルミーンフィルタのパラメータを指定
h_p = 7

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
output_file_skel_junk = "../Output/Remove_Background/ORA-{0:03d}_NLMD_h{1:d}_skel_junk.png".format(img_num,h_p)

#画像の入力
img = cv2.imread(input_file,0)
img_col = cv2.imread(input_file)

#ノンローカルミーンフィルタ
img_NLMD = cv2.fastNlMeansDenoising(img,h=h_p)

#フィルタ結果を2値化
th, th_img = cv2.threshold(img_NLMD, 0, 255, cv2.THRESH_OTSU)
th_img = np.where(th_img==255 ,1 ,0)

# 細線化(スケルトン化) Zha84
skeleton = skeletonize(th_img)

skel_b = np.zeros((img.shape[0],img.shape[1]))
skel_b[skeleton==True]=255

temp,img_j = junk(skel_b,img_col)
cv2.imwrite(output_file_skel_junk,img_j)

In [ ]:
def junk(img_skel,img):
    height,width = img_skel.shape
    temp = np.zeros((height,width))
    
    for y in range(0,height):
        for x in range(0,width):
            if(img_skel[y,x]>200):
                check = 0
                check_1 = 0
                cnt = 1
                temp_l = np.zeros((3,3))
                
                if(img_skel[y-1,x-1] > 200):
                    cnt += 1
                    temp_l[0,0] = 1
                    #if(temp[y-1,x-1]==1):
                    #    check = 1
                if(img_skel[y-1,x] > 200):
                    cnt += 1
                    temp_l[0,1] = 1
                    #if(temp[y-1,x]==1):
                    #    check = 1
                if(img_skel[y-1,x+1] > 200):
                    cnt += 1
                    temp_l[0,2] = 1
                    #if(temp[y-1,x+1]==1):
                    #    check = 1
                if(img_skel[y,x-1] > 200):
                    cnt += 1
                    temp_l[1,0] = 1
                    #if(temp[y,x-1]==1):
                    #    check = 1
                if(img_skel[y,x+1] > 200):
                    cnt += 1
                    temp_l[1,2] = 1
                    #if(temp[y,x+1]==1):
                    #    check = 1
                if(img_skel[y+1,x-1] > 200):
                    cnt += 1
                    temp_l[2,0] = 1
                    #if(temp[y+1,x-1]==1):
                    #    check = 1
                if(img_skel[y+1,x] > 200):
                    cnt += 1
                    temp_l[2,1] = 1
                    #if(temp[y+1,x]==1):
                    #    check = 1
                if(img_skel[y+1,x+1] > 200):
                    cnt += 1
                    temp_l[2,2] = 1
                    #if(temp[y-1,x-1]==1):
                    #    check = 1
                
                for y_l in range(3):
                    for x_l in range(3):
                        if(temp_l[y_l,x_l] == 1):
                            if((y_l-1)>=0):
                                if(temp_l[y_l-1,x_l] == 1):
                                    check_1 = 1
                            if((y_l+1)<3):
                                if(temp_l[y_l+1,x_l] == 1):
                                    check_1 = 1
                            if((x_l-1)>=0):
                                if(temp_l[y_l,x_l-1] == 1):
                                    check_1 = 1
                            if((x_l+1)<3):
                                if(temp_l[y_l,x_l+1] == 1):
                                    check_1 = 1

                if(cnt == 2):
                    temp[y,x] = 2
                elif((cnt > 3) and (check == 0) and (check_1 == 0)):
                    temp[y,x] = 1
                    
    for y in range(height):
        for x in range(width):
            if(temp[y,x]==1):
                cv2.rectangle(img, (x-3, y-3), (x+3, y+3), (0, 0, 255))
            elif(temp[y,x]==2):
                cv2.rectangle(img, (x-3, y-3), (x+3, y+3), (0, 255, 0))
                    
    return temp,img

### 分岐点検出用のデータセット

In [ ]:
def junk_dataset(img_skel,img,img_num,img_gray):
    height,width = img_skel.shape
    temp = np.zeros((height,width))
    
    for y in range(0,height):
        for x in range(0,width):
            if(img_skel[y,x]>200):
                check = 0
                check_1 = 0
                cnt = 1
                temp_l = np.zeros((3,3))
                
                if(img_skel[y-1,x-1] > 200):
                    cnt += 1
                    temp_l[0,0] = 1
                    #if(temp[y-1,x-1]==1):
                    #    check = 1
                if(img_skel[y-1,x] > 200):
                    cnt += 1
                    temp_l[0,1] = 1
                    #if(temp[y-1,x]==1):
                    #    check = 1
                if(img_skel[y-1,x+1] > 200):
                    cnt += 1
                    temp_l[0,2] = 1
                    #if(temp[y-1,x+1]==1):
                    #    check = 1
                if(img_skel[y,x-1] > 200):
                    cnt += 1
                    temp_l[1,0] = 1
                    #if(temp[y,x-1]==1):
                    #    check = 1
                if(img_skel[y,x+1] > 200):
                    cnt += 1
                    temp_l[1,2] = 1
                    #if(temp[y,x+1]==1):
                    #    check = 1
                if(img_skel[y+1,x-1] > 200):
                    cnt += 1
                    temp_l[2,0] = 1
                    #if(temp[y+1,x-1]==1):
                    #    check = 1
                if(img_skel[y+1,x] > 200):
                    cnt += 1
                    temp_l[2,1] = 1
                    #if(temp[y+1,x]==1):
                    #    check = 1
                if(img_skel[y+1,x+1] > 200):
                    cnt += 1
                    temp_l[2,2] = 1
                    #if(temp[y-1,x-1]==1):
                    #    check = 1
                
                for y_l in range(3):
                    for x_l in range(3):
                        if(temp_l[y_l,x_l] == 1):
                            if((y_l-1)>=0):
                                if(temp_l[y_l-1,x_l] == 1):
                                    check_1 = 1
                            if((y_l+1)<3):
                                if(temp_l[y_l+1,x_l] == 1):
                                    check_1 = 1
                            if((x_l-1)>=0):
                                if(temp_l[y_l,x_l-1] == 1):
                                    check_1 = 1
                            if((x_l+1)<3):
                                if(temp_l[y_l,x_l+1] == 1):
                                    check_1 = 1

                if(cnt == 2):
                    temp[y,x] = 2
                elif((cnt > 3) and (check == 0) and (check_1 == 0)):
                    temp[y,x] = 1
                    
    for y in range(height):
        for x in range(width):
            if(temp[y,x]==1):
                cv2.rectangle(img, (x-3, y-3), (x+3, y+3), (0, 0, 255))
            elif(temp[y,x]==2):
                cv2.rectangle(img, (x-3, y-3), (x+3, y+3), (0, 255, 0))
    
    cv2.imwrite('Output/junk_reserch/junk_check/ORA-{0:03d}_NLMD-otsu_h5_skel_junk.png'.format(img_num),img)
                
    for y in range(height):
        for x in range(width):
            if(temp[y,x]==1):
                img_crop = img_gray[y-15:y+15,x-15:x+15]
                cv2.imwrite('Output/junk_reserch/junk_crop/ORA-{0:03d}_NLMD-otsu_h5_skel_junk_{1:03d}-{2:03d}.png'.format(img_num,y,x),img_crop)
    

In [ ]:
img_num = 500

img = cv2.imread('../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif'.format(img_num),0)
img_col = cv2.imread('../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif'.format(img_num))

img_NLMD = cv2.fastNlMeansDenoising(img,h=5)

th, th_img = cv2.threshold(img_NLMD, 0, 255, cv2.THRESH_OTSU)
th_img = np.where(th_img==255 ,1 ,0)

# 細線化(スケルトン化) Zha84
skeleton = skeletonize(th_img)

print(skeleton)

skel_b = np.zeros((img.shape[0],img.shape[1]))
skel_b[skeleton==True]=255

junk_dataset(skel_b,img_col,img_num,img)

### ガボールフィルタのパラメータ調整

In [ ]:
from ipywidgets import interact

@interact
def test(k:(1,50,1),sigma:(0,25,0.01),deg:(0,179,1),lam:(0,200,0.01),gam:(0,20,0.01)):
    
    #画像番号の指定
    img_num = 661

    #入力のファイルを指定
    input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
    
    #画像の入力
    img = cv2.imread(input_file,0)
    
    #ガボールフィルタ
    #gabor = cv2.getGaborKernel((k,k), sigma, np.radians(deg), lam, gam, 0)

    '''
    gabor = cv2.getGaborKernel((30,30), 1.11, np.radians(30), 10, 0.09, 0)    
    #gabor_w3 = cv2.getGaborKernel((20,20), 1, np.radians(30), 10, 0.07, 0)
    #gabor_w5 = cv2.getGaborKernel((20,20), 2.2, np.radians(deg), 30, 0.1, 0)
    #gabor_w7 = cv2.getGaborKernel((20,20), 2.6, np.radians(30), 30, 0.1, 0)
    #gabor_w9 = cv2.getGaborKernel((20,20), 5, np.radians(30), 30, 0.15, 0)
    '''
    
    #ガボールフィルタ適用
    dst = cv2.filter2D(img_col, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)
    print(deg,k,sigma,lam,gam)
    
    fig = plt.figure()
    plt.figure(figsize=(10, 10))
    
    #表示
    #plt.imshow(dst,cmap="jet")
    plt.imshow(gabor,cmap="jet")
    
    #plt.imshow(np.array(dst))
    #fig.savefig('Output/gabor_test/gabor_k20_sig5_deg30_lam30_gam0-15.png')
    #np.savetxt('Output/gabor_test/ORA-661_gabor_k30_sig1-11_deg{0:d}_lam10_gam0-09.csv'.format(deg), dst, delimiter=',')

### k-meansによる背景削除

In [ ]:
#画像番号の指定
img_num = 630

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
output_file = "../Output/Remove_Background/ORA-{0:03d}_G1-5-5_k-means_del_back.png".format(img_num)

#画像の入力
img = cv2.imread(input_file,0)
img_col = cv2.imread(input_file)

height,width = img.shape

#ガウシアンフィルタ
img = cv2.GaussianBlur(img,(5,5),1)

img = img.reshape(-1,1)

pred = KMeans(n_clusters=2).fit_predict(img)

i = 0
for y in range(height):
    for x in range(width):
        if(pred[i]==1):
            img_col[y,x]=(0,0,255)
        i += 1
        
cv2.imwrite(output_file,img_col)

### 適応的閾値処理

In [ ]:
#画像番号の指定
img_num = 500

#ノンローカルミーンフィルタのパラメータを指定
h_p = 6

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
#output_file = "reference_output/ORA-{0:03d}_adaptive-th.png".format(img_num)
output_file_NLMD= "reference_output/ORA-{0:03d}_NLMD-h{1:d}_adaptive-th.png".format(img_num,h_p)

#画像の入力
img = cv2.imread(input_file,0)

#ノンローカルミーンフィルタ
img_NLMD = cv2.fastNlMeansDenoising(img,h=h_p)
th = 50
img_fore = np.where(img_NLMD > th,img_NLMD,0)
th_img = np.where(img_NLMD > th, 255, 0)

#適応的閾値処理
dst = cv2.adaptiveThreshold(img_fore,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,3,2)

temp = th_img - dst

cv2.imwrite(output_file_NLMD,temp)

### 細線化画像を用いた花弁の幅探索

In [ ]:
#画像番号の指定
img_num = 661

#ノンローカルミーンフィルタのパラメータを指定
h_p = 7

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
output_file_skel_junk = "../Output/Remove_Background/ORA-{0:03d}_NLMD_h{1:d}_skel_junk.png".format(img_num,h_p)

#画像の入力
img = cv2.imread(input_file,0)
img_col = cv2.imread(input_file)

height,width = img.shape

#ノンローカルミーンフィルタ
img_NLMD = cv2.fastNlMeansDenoising(img,h=h_p)

#フィルタ結果を2値化
th, th_img = cv2.threshold(img_NLMD, 0, 255, cv2.THRESH_OTSU)
th_img = np.where(th_img==255 ,1 ,0)

# 細線化(スケルトン化) Zha84
skeleton = skeletonize(th_img)

skel_b = np.zeros((img.shape[0],img.shape[1]))
skel_b[skeleton==True]=255

for y  in tqdm(range(height)):
    for x in range(width):
        if(skel_b[y,x]==255):
            

### Canny法

In [ ]:
#画像番号の指定
img_num = 661

#ノンローカルミーンフィルタのパラメータを指定
h_p = 7

#入力のファイルを指定
input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
#出力のファイルを指定
output_file_skel_junk = "../Output/Remove_Background/ORA-{0:03d}_canny.png".format(img_num,h_p)

#画像の入力
img = cv2.imread(input_file,0)
img_col = cv2.imread(input_file)

In [ ]:
from ipywidgets import interact

@interact
def test(min_th:(50,500,1),max_th:(50,500,1)):
    
    #画像番号の指定
    img_num = 300

    #入力のファイルを指定
    input_file = "../Input/ORA-Coronal_y_part/ORA-Coronal-{0:03d}_part.png".format(img_num)
    
    #画像の入力
    img = cv2.imread(input_file,0)
    
    #ノンローカルミーンフィルタ
    img_NLMD = cv2.fastNlMeansDenoising(img,h=5)
    
    img_NLMD = np.where(img_NLMD>50 ,img_NLMD,0)
    
    print(min_th,max_th)
    dst = cv2.Canny(img_NLMD, min_th, max_th)
    
    fig = plt.figure()
    plt.figure(figsize=(30, 30))
    
    #表示
    plt.imshow(dst,cmap="gray")

In [ ]:
from ipywidgets import interact

@interact
def test(min_th:(0,255,1),max_th:(0,255,1)):
    
    #画像番号の指定
    img_num = 300

    #入力のファイルを指定
    input_file = "../Input/ORA-Coronal_y_part/ORA-Coronal-{0:03d}_part.png".format(img_num)
    
    #画像の入力
    img = cv2.imread(input_file,0)
    
    #ノンローカルミーンフィルタ
    img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
    
    img_NLMD = np.where(img_NLMD > min_th ,img_NLMD,500)
    img_NLMD = np.where(img_NLMD < max_th ,255,0)
    
    print(min_th,max_th)
    
    fig = plt.figure()
    plt.figure(figsize=(30, 30))
    
    #表示
    plt.imshow(img_NLMD,cmap="gray")

### CT画像を縦横全て出力

In [ ]:
temp = 600
#入力のファイルを指定
temp_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(temp)

#画像の入力
img_temp = cv2.imread(temp_file,0)

height,width = img_temp.shape

Volume = np.zeros((888,height,width))

#ボリュームデータの格納
for z in tqdm(range(888)):
    input_file = "../Input/ORA-PNG/ORA-{0:03d}.png".format(z)
    img_num = 887 - z
    Volume[img_num,:,:] = cv2.imread(input_file,0)
    
#Sagittal_x
for x in tqdm(range(width)):
    output_file = "../Input/ORA-Sagittal_x/ORA-Sagittal-{0:03d}.png".format(x)
    img = Volume[:,:,x]
    
    cv2.imwrite(output_file,img)

#Coronal_y
for y in tqdm(range(height)):
    output_file = "../Input/ORA-Coronal_y/ORA-Coronal-{0:03d}.png".format(y)
    img = Volume[:,y,:]
    
    cv2.imwrite(output_file,img)

### 冠状面のCT画像を一部切り取り

In [ ]:
import numpy as np
from tqdm import tqdm

img_num = 600

temp = "../Output/Spectral_sys/12_junk-del-label-inte-sp/ORA-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
img_c = cv2.imread(temp,0)

height,width = img_c.shape

Volume = np.zeros((888,height,width))

for z in tqdm(range(888)):
    input_file = "Input/ORA-PNG/ORA-{0:03d}.png".format(z)
    img_num = 887 - z
    img = cv2.imread(input_file,0)
    Volume[img_num,:,:] = img[:,:]
    
for y in tqdm(range(height)):
    img = Volume[0:500,y,:]

    cv2.imwrite('../Input/ORA-Coronal_y_part/ORA-Coronal-{0:03d}_part.png'.format(y),img)
    #cv2.imwrite('Output/3D_inte/3D-inte_total/ORA-Coronal-{0:03d}_inte.png'.format(y),img)